In [1]:
import tensorflow as tf


2023-03-26 23:49:59.937534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 23:50:00.921739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [59]:
BASE_PATH = '/home/ec2-user/isolated-sign-language-recognition/sample-data'

In [4]:
index = pd.read_csv('index.csv')
# sample_data = pd.read_csv(BASE_PATH)

In [6]:
index = index.drop(columns='path')
index.head()

,participant_id,sequence_id,sign
0,26734,1000035562,blow
1,28656,1000106739,wait
2,16069,100015657,cloud
3,25571,1000210073,bird
4,62590,1000240708,owie


In [32]:
class_labels = index.sign.unique()
class_labels

array(['blow', 'wait', 'cloud', 'bird', 'owie', 'duck', 'minemy', 'lips',
       'flower', 'time', 'vacuum', 'apple', 'puzzle', 'mitten', 'there',
       'dry', 'shirt', 'owl', 'yellow', 'not', 'zipper', 'clean',
       'closet', 'quiet', 'have', 'brother', 'clown', 'cheek', 'cute',
       'store', 'shoe', 'wet', 'see', 'empty', 'fall', 'balloon',
       'frenchfries', 'finger', 'same', 'cry', 'hungry', 'orange', 'milk',
       'go', 'drawer', 'TV', 'another', 'giraffe', 'wake', 'bee', 'bad',
       'can', 'say', 'callonphone', 'finish', 'old', 'backyard', 'sick',
       'look', 'that', 'black', 'yourself', 'open', 'alligator', 'moon',
       'find', 'pizza', 'shhh', 'fast', 'jacket', 'scissors', 'now',
       'man', 'sticky', 'jump', 'sleep', 'sun', 'first', 'grass', 'uncle',
       'fish', 'cowboy', 'snow', 'dryer', 'green', 'bug', 'nap', 'feet',
       'yucky', 'morning', 'sad', 'face', 'penny', 'gift', 'night',
       'hair', 'who', 'think', 'brown', 'mad', 'bed', 'drink', 'stay',


In [71]:
X_train, X_test, y_train, y_test = train_test_split(index[['participant_id','sequence_id']], index['sign'], 
                                                    test_size=0.2, stratify=index['sign'])

train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)


In [157]:
import glob

def get_train_sample(train_df):
        
    for index in train_df.index:
        metadata = train_df.loc[index]
        files = glob.glob(f"{BASE_PATH}/participant_id={metadata['participant_id']}/sequence_id={metadata['sequence_id']}/*.csv")
        df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)
        
        df = df.sort_values('frame').values.astype(np.float32)
        
        yield df, metadata['sign']
                
        # yield df.sort_values('frame').to_numpy()
        # yield (df.reshape(-1), 0.0)
        # yield (df.reshape(-1), metadata['sign'])        
    
        
def get_train_sample(train_df):
        
    for index in train_df.index:
        metadata = train_df.loc[index]
        files = f"{BASE_PATH}/participant_id={metadata['participant_id']}/sequence_id={metadata['sequence_id']}/*.csv"
        
        yield tf.convert_to_tensor(files,dtype=tf.string), tf.convert_to_tensor(metadata['sign'],dtype=tf.string)        
    

In [158]:
tmp_df = index[(index['participant_id']==16069) & (index['sequence_id']==1004211348)]
# tmp_df = index[(index['participant_id']==16069) & (index['sequence_id']==1005492440)]

In [159]:
# tf.convert_to_tensor(next(get_train_sample(tmp_df)))
next(get_train_sample(tmp_df))

(<tf.Tensor: shape=(), dtype=string, numpy=b'/home/ec2-user/isolated-sign-language-recognition/sample-data/participant_id=16069/sequence_id=1004211348/*.csv'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'bug'>)

In [149]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [160]:
dataset = tf.data.Dataset.from_generator(get_train_sample, args=[tmp_df],
                                         output_types=tf.string)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).